9. Haciendo uso de sockets, implemente un servidor y un cliente (a modo de ejemplo, se proveen un servidor y un cliente en lenguaje Python), que permita desde el cliente, enviar un archivo comprimido utilizando el alfabeto ABCDEFGH, y en el servidor, descomprimir el archivo

Este programa implementa un **servidor TCP en Python** que recibe datos comprimidos usando Huffman y los descomprime.  
Se encarga de reconstruir el texto original y guardarlo en un archivo `descomprimido.txt`.

In [1]:
import socket
import ast

La función `descomprimir` recibe:
- `codigo_bin`: el texto comprimido en binario.
- `tabla`: la tabla de Huffman enviada por el cliente.

Proceso:
1. Invierte la tabla para buscar códigos de manera rápida.
2. Recorre el código binario acumulando bits.
3. Cada vez que coincide un código, se agrega el carácter correspondiente al resultado.

In [2]:
def descomprimir(codigo_bin, tabla):
    # Invertimos la tabla para búsqueda rápida
    inv_tabla = {v: chr(k) for k, v in tabla.items()}
    actual = ""
    resultado = ""
    for bit in codigo_bin:
        actual += bit
        if actual in inv_tabla:
            resultado += inv_tabla[actual]
            actual = ""
    return resultado

En esta sección se configura el servidor:
- IP: `0.0.0.0` (acepta conexiones desde cualquier interfaz)
- Puerto: `5555`  

El servidor:
1. Espera a que un cliente se conecte.
2. Recibe los datos enviados: primero la tabla de Huffman y luego el texto comprimido.
3. Separa la tabla del código y la convierte de cadena a diccionario.
4. Descomprime el texto usando la función `descomprimir`.
5. Imprime la información recibida y guarda el resultado en `descomprimido.txt`.
6. Cierra la conexión con el cliente.

In [3]:
ip = "0.0.0.0"
puerto = 5555

servidor = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
servidor.bind((ip, puerto))
servidor.listen(1)

print(f"[*] Esperando conexiones en {ip}:{puerto}")

cliente, direccion = servidor.accept()
print(f"[*] Conexión establecida con {direccion[0]}:{direccion[1]}")

# Recibir datos
datos = cliente.recv(8192).decode("utf-8")

# Separar tabla y código
tabla_str, comprimido = datos.split("\n", 1)
tabla = ast.literal_eval(tabla_str)

print("[*] Tabla recibida:", tabla)
print("[*] Texto comprimido recibido:", comprimido)

# Descomprimir
texto_descomprimido = descomprimir(comprimido, tabla)

print("[*] Texto descomprimido:", texto_descomprimido)

# Guardar en archivo
with open("descomprimido.txt", "w") as f:
    f.write(texto_descomprimido)

cliente.close()
servidor.close()


[*] Esperando conexiones en 0.0.0.0:5555
[*] Conexión establecida con 127.0.0.1:52259
[*] Tabla recibida: {65: '0', 66: '10', 67: '1100', 72: '11010', 68: '11011', 69: '1110', 71: '11110', 70: '11111'}
[*] Texto comprimido recibido: 000010101011001100110110011101110111111111011010000101010
[*] Texto descomprimido: AAAABBBCCDAAEEFGHAAABBB
